# 1. Notebook Bouwmaterialen.ipynb

# Doel en context

Dit notebook heeft als doel om het aantal materialen die een funnctie heeft te weergeven. Dit is essentieel voor het project, omdat hierin staat uit welke functies welke materialen gehaald kunnen worden voor nieuwbouw.

# Achtergrond van het probleem

De materialen van oude gebouwen worden vaak weggegooid, terwijl dezevaak nog hergebruikt kan worden. Door het beter te documenteren van bouwmaterialen in deze functies, kunnen de bouwmaterialen gerecycled worden voor nieuwbouw.

# Stap 1. Datasets inladen

In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\niels\Downloads\Heerlen_dataset.csv")
df

df = pd.read_csv(r"C:\Users\niels\OneDrive\Documents\materiaal.csv")
df


,functie,bouwjaar,beton,overige constructiematerialen,baksteen,staal,hout,isolatie,keramiek,glas
0,vrijstaand,<19445,975,250,200,25,100,10,10,0
1,vrijstaand,1945-1970,1025,250,200,10,50,10,10,0
2,vrijstaand,1970-2000,1025,1550,75,25,25,10,0,0
3,vrijstaand,>2000,1025,1550,75,25,25,10,0,0
4,serieel,<1945,800,200,100,25,25,10,0,0
5,serieel,1945-1970,850,200,100,25,25,10,0,0
6,serieel,1970-2000,900,275,25,25,25,10,0,0
7,serieel,>2000,900,275,25,25,25,10,0,0
8,appartement,<1945,1400,250,175,50,10,25,0,0
9,appartement,1945-1970,1450,250,175,50,10,25,0,0


Deze tabel is gemaakt aan de hand van een tabel uit https://www.cirkelregio-utrecht.nl/wp-content/uploads/2022/12/universiteit_leiden.pdf 

# Stap 2. Materiaalhoeveelheden Berekenen

In [2]:
df_heerlen = pd.read_csv(r"C:\Users\niels\Downloads\Heerlen_dataset.csv")
df_materiaal = pd.read_csv(r"C:\Users\niels\OneDrive\Documents\materiaal.csv")

def bereken_materialen_per_adres(data_heerlen_content, materiaal_content, postcode, huisnummer):
    try:
        df_pand = df.copy()
        df_pand['huisnummer'] = pd.to_numeric(df_pand['huisnummer'], errors='coerce')
        if 'postcode' in df_pand.columns:
            df_pand['postcode'] = df_pand['postcode'].astype(str).str.replace(' ', '').str.upper()
        df_materiaal['functie'] = df_materiaal['functie'].str.lower().str.strip()
    except Exception as e:
        return {"fout": f"Fout bij het verwerken van de dataframes: {e}"}

    try:
        huisnummer = int(huisnummer)
    except ValueError:
        return {"fout": "Huisnummer moet een geldig getal zijn."}

    postcode_formatted = postcode.replace(' ', '').upper()
    gefilterd_pand = df_pand[
        (df_pand['postcode'] == postcode_formatted) &
        (df_pand['huisnummer'] == huisnummer)
    ]

    if gefilterd_pand.empty:
        return {"resultaat": f"Gebouw niet gevonden op basis van postcode '{postcode}' en huisnummer '{huisnummer}'."}

    pand_info = gefilterd_pand.iloc[0]
    bouwjaar = pand_info.get('bouwjaar')
    oppervlakte = pand_info.get('opp_pand')
    if pd.isna(oppervlakte) or oppervlakte <= 0:
        oppervlakte = pand_info.get('opp_adresseerbaarobject_m2')

    if pd.isna(bouwjaar) or pd.isna(oppervlakte) or oppervlakte <= 0:
        return {"resultaat": "Gegevens ontbreken (bouwjaar of oppervlakte zijn niet beschikbaar of ongeldig)."}

    functie_mapping = {
        'vrijstaand': ['vrijstaande woning', 'tweeonder1kap'],
        'serieel': ['tussen of geschakelde woning', 'hoekwoning'],
        'appartement': ['appartement'],
        'winkel': ['winkelfunctie'],
        'kantoor': ['kantoorfunctie'],
        'bedrijfshal': ['industriefunctie'],
        'zorg': ['gezondheidszorgfunctie'],
        'onderwijs': ['onderwijsfunctie']
    }

    gevonden_functie_categorie = None
    woningtype_uit_data = str(pand_info.get('woningtype', '')).lower().strip()
    for categorie, types in functie_mapping.items():
        if categorie in ['vrijstaand', 'serieel', 'appartement']:
            if woningtype_uit_data in [t.lower().strip() for t in types]:
                gevonden_functie_categorie = categorie
                break

    if gevonden_functie_categorie is None:
        if pand_info.get('winkelfunctie') == 1:
            gevonden_functie_categorie = 'winkel'
        elif pand_info.get('kantoorfunctie') == 1:
            gevonden_functie_categorie = 'kantoor'
        elif pand_info.get('industriefunctie') == 1:
            gevonden_functie_categorie = 'bedrijfshal'
        elif pand_info.get('gezondheidszorgfunctie') == 1:
            gevonden_functie_categorie = 'zorg'
        elif pand_info.get('onderwijsfunctie') == 1:
            gevonden_functie_categorie = 'onderwijs'

    if gevonden_functie_categorie is None:
        return {"resultaat": "Gegevens ontbreken (functie van het gebouw kon niet worden vastgesteld of gekoppeld aan de materialentabel)."}

    if bouwjaar < 1945:
        bouwjaar_interval = '<1945'
    elif 1945 <= bouwjaar <= 1970:
        bouwjaar_interval = '1945-1970'
    elif 1970 < bouwjaar <= 2000:
        bouwjaar_interval = '1970-2000'
    elif bouwjaar > 2000:
        bouwjaar_interval = '>2000'
    else:
        return {"resultaat": "Gegevens ontbreken (bouwjaar interval kon niet worden bepaald)."}

    materiaal_rij = df_materiaal[
        (df_materiaal['functie'] == gevonden_functie_categorie) &
        (df_materiaal['bouwjaar'] == bouwjaar_interval)
    ]

    if materiaal_rij.empty:
        return {"resultaat": f"Gegevens ontbreken (geen materiaalinformatie gevonden voor functie '{gevonden_functie_categorie}' en bouwjaar '{bouwjaar_interval}')."}

    materiaal_gewichten = materiaal_rij.iloc[0].drop(['functie', 'bouwjaar']).astype(float)
    berekende_materialen = {materiaal: round(gewicht_per_m2 * oppervlakte, 2) if pd.notna(gewicht_per_m2) else 0 for materiaal, gewicht_per_m2 in materiaal_gewichten.items()}

    return {
        "resultaat": berekende_materialen,
        "info_pand": pand_info.to_dict(),
        "oppervlakte": oppervlakte,
        "functie_categorie": gevonden_functie_categorie,
        "bouwjaar_interval": bouwjaar_interval
    }

# Uitleg van de Code±
De functie bereken_materialen_per_adres is ontwikkeld om de verwachte hoeveelheden materialen voor een pand in Heerlen te schatten, gebaseerd op postcode, huisnummer, bouwjaar en type gebouw. Deze functie maakt gebruik van twee ingevoerde dataframes: df_heerlen, dat informatie over gebouwen in Heerlen bevat, en df_materiaal, een tabellenlijst met materiaalgewichten per vierkante meter, ingedeeld op basis van gebouwfunctie en periodes van bouwjaren.

1. Gegevensvoorbereiding en Validatie
Een exemplaar van het df_heerlen-dataframe wordt aangemaakt als df_pand om de oorspronkelijke informatie te behouden. De kolom 'huisnummer' binnen df_pand wordt omgezet naar een numeriek formaat, waarbij eventuele fouten worden vervangen door NaN. Als de kolom 'postcode' aanwezig is, wordt deze omgevormd naar hoofdletters en worden eventuele spaties verwijderd voor een uniforme vergelijking. De kolom 'functie' in df_materiaal wordt omgezet naar kleine letters en overtollige spaties worden weggehaald. Basis foutafhandeling is toegevoegd voor mogelijke complicaties bij de verwerking van gegevens. Het ingevoerde huisnummer wordt omgezet naar een integer; bij een ongeldige invoer verschijnt er een foutmelding. De ingevoerde postcode wordt aangepast om overeen te komen met het format in df_pand.

2. Ophalen van Gebouwinformatie
Het df_pand-dataframe wordt gefilterd om de rij te lokaliseren die overeenkomt met de opgegeven postcode en het huisnummer. Als er geen gebouw kan worden gevonden, wordt er een waarschuwingsbericht verzonden. Het bouwjaar en de oppervlakte worden gedestilleerd uit de informatie van het aangetroffen gebouw. Als de oppervlakte ontbreekt of ongeldig is, wordt gepoogd om de waarde uit opp_adresseerbaarobject_m2 te gebruiken. Er worden controles uitgevoerd om te verzekeren dat het bouwjaar en de oppervlakte geldig zijn; zoniet wordt er een melding over ontbrekende gegevens teruggestuurd.

3. Categorisering van Gebouwfunctie
Een functie_mapping-woordenboek specificeert categorieën zoals 'vrijstaand', 'serieel', 'appartement', 'winkel', 'kantoor', 'bedrijfshal', 'zorg' en 'onderwijs'. Deze categorieën worden in verband gebracht met verschillende woningtype-waarden of specifieke functie-indicatoren (bijvoorbeeld winkelfunctie). De code probeert aanvankelijk de functie van het gebouw te identificeren aan de hand van het woningtype. Indien er geen residentieel woningtype wordt aangetroffen, worden commerciële of publieke functie-indicatoren (bijvoorbeeld winkelfunctie == 1) nagekeken. Als de functie niet kan worden bepaald, verschijnt er een foutmelding.

4. Vaststelling Bouwjaarinterval
Het bouwjaar wordt ingedeeld in vooraf gedefinieerde categorieën: '2000'.
Indien het niet mogelijk is om het bouwjaarinterval vast te stellen, wordt er een foutbericht gecreëerd.

5. Materiaalschatting
Het df_materiaal-dataframe wordt gefilterd om de rij te vinden die overeenkomt met de vastgestelde gevonden_functie_categorie en het bouwjaar_interval. Als er geen informatie over materiaal beschikbaar is voor de gegeven functie en het jaarinterval, wordt er een gepast bericht verzonden. De materiaalgewichten per vierkante meter worden uit de relevante rij in df_materiaal gehaald, met uitzondering van de kolommen 'functie' en 'bouwjaar', en worden omgezet naar float. Het berekende_materialen-woordenboek wordt samengesteld door het gewicht per vierkante meter van elk materiaal te vermenigvuldigen met de oppervlakte van het gebouw, afgerond op twee decimalen.

6. Resultaat Retourneren
De functie levert een woordenboek op met de volgende onderdelen:
resultaat: Een woordenboek met de berekende hoeveelheden materialen (in kilogram) voor het gebouw.
info_pand: Een woordenboek met alle relevante gegevens over het gevonden gebouw.
oppervlakte: De oppervlakte van het gebouw die gebruikt is voor de berekening.
functie_categorie: De vastgestelde functiecategorie van het gebouw.
bouwjaar_interval: Het vastgestelde bouwjaarinterval van het gebouw.

# Stap 3. Interactieve materiaalvisualisatie

In [3]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets

# Widgets maken voor invoer
postcode_input = widgets.Text(description="Postcode:", placeholder="Bijv. 6411AB")
huisnummer_input = widgets.Text(description="Huisnummer:", placeholder="Bijv. 12")
button = widgets.Button(description="Toon materialen")
uitvoer = widgets.Output()

# Functie voor button click
def on_button_click(b):
    with uitvoer:
        clear_output()
        
        postcode_str = postcode_input.value.strip()
        huisnummer_str = huisnummer_input.value.strip()
        
        if not postcode_str:
            print("❌ Vul een postcode in.")
            return
        if not huisnummer_str:
            print("❌ Vul een huisnummer in.")
            return
        
        try:
            huisnummer_int = int(huisnummer_str)
        except ValueError:
            print("❌ Huisnummer moet een geldig getal zijn.")
            return
        
        # Zorg dat je hier jouw eigen functie gebruikt!
        res = bereken_materialen_per_adres(df_heerlen, df_materiaal, postcode_str, huisnummer_int)
        
        if "fout" in res:
            print("❌ Fout:", res["fout"])
            return
        
        if "resultaat" in res:
            # Als resultaat een dict is met materiaaldata
            if isinstance(res["resultaat"], dict) and res["resultaat"]:
                materialen_dict = res["resultaat"]
                
                materialen = list(materialen_dict.keys())
                hoeveelheden = list(materialen_dict.values())
                
                plt.figure(figsize=(10,5))
                bars = plt.bar(materialen, hoeveelheden, color='steelblue')
                plt.title(f"Materialen - {res.get('functie_categorie', '')} ({res.get('bouwjaar_interval', '')})")
                plt.ylabel("Hoeveelheid (kg)")
                plt.xticks(rotation=45, ha='right')
                plt.grid(axis='y', linestyle='--', alpha=0.7)
                
                for bar in bars:
                    yval = bar.get_height()
                    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 1),
                             ha='center', va='bottom', fontsize=8)
                plt.tight_layout()
                plt.show()
            else:
                # Geen materiaaldata gevonden, toon melding
                print("ℹ️ Materiaaldata nog niet beschikbaar voor dit adres.")
        else:
            print("⚠️ Onverwachte output van de functie.")

button.on_click(on_button_click)

display(postcode_input, huisnummer_input, button, uitvoer)

Text(value='', description='Postcode:', placeholder='Bijv. 6411AB')

Text(value='', description='Huisnummer:', placeholder='Bijv. 12')

Button(description='Toon materialen', style=ButtonStyle())

Output()

# Uitleg van de Code

De aangeboden code ontwikkelt een interactieve omgeving via ipywidgets voor het in beeld brengen van materialen. Hierdoor kunnen gebruikers hun postcode en huisnummer invoeren, waarna de verwachte materialen voor dat adres zichtbaar worden in de vorm van een staafdiagram.
De opbouw van de code-functionaliteit is als volgt:
1. Widget-initialisatie:
postcode_input en huisnummer_input: Dit zijn invoervelden (widgets. Text) waarin de gebruiker een postcode en een huisnummer kan invoeren. Plaatsaanduidingen (placeholder) en uitleg (description) zijn toegevoegd voor een betere gebruikerservaring. button: Een knop (widgets. Button) met de tekst "Toon materialen". Deze knop start de berekening en de visualisatie. uitvoer: Een uitvoergebied (widgets. Output) waar berichten en het gemaakte staafdiagram worden getoond.
2. on_button_click Functie:
Deze functie gaat van start wanneer er op de knop "Toon materialen" wordt geklikt. In het uitvoer-gebied wordt eerst de vorige uitvoer gewist (clear_output()) om ruimte te maken voor nieuwe resultaten. De waarden voor postcode en huisnummer worden uit de widgets gehaald en eventuele extra spaties worden verwijderd (strip()). Validatie: Er wordt gecontroleerd of de invoervelden gevuld zijn en of het huisnummer in het juiste numerieke formaat is. Bij onjuiste invoer wordt een foutmelding gegeven Functieaanroep: Het belangrijkste deel van deze functie is het aanroepen van bereken_materialen_per_adres(), waarbij df_heerlen en df_materiaal (ervan uitgaande dat deze in de omgeving zijn) samen met de ingevoerde postcode en het omgezette huisnummer als argumenten worden meegegeven.
3. Resultaatverwerking:
Als de bereken_materialen_per_adres-functie een "fout"-sleutel in de reactie heeft, wordt de foutmelding weergegeven. Als de reactie een "resultaat"-sleutel bevat en dit een niet-lege dictionary is (wat aangeeft dat de materiaaldatabase succesvol is berekend), wordt er een staafdiagram aangemaakt: De materiaalsoorten en hun hoeveelheden (in kg) worden verzameld. matplotlib. pyplot wordt ingezet om een staafdiagram te tekenen, met de materialen op de x-as en hun hoeveelheden op de y-as. De grafiek krijgt een titel die de functie en het bouwjaar van het gebouw aangeeft. De labels op de x-as worden gedraaid en er wordt een grid ingezorgd voor een betere leesbaarheid. De hoeveelheid waarden worden als tekst boven de staven geplaatst.
De grafiek wordt getoond (plt. show()). Als "resultaat" geen materiaalgegevens heeft, verschijnt een informatieve melding dat er nog geen materiaalinformatie beschikbaar is. Bij onverwachte uitvoer van de functie wordt een algemene waarschuwing gegeven.
4. Event-binding en Display:
button. on_click(on_button_click): Dit verbindt de on_button_click-functie aan de klikactie van de knop, zodat deze functie activeert bij het indrukken van de knop. display(. . . ): Deze regel zorgt ervoor dat de widgets (postcode-invoer, huisnummer-invoer, knop en uitvoergebied) daadwerkelijk zichtbaar zijn binnen de Jupyter-omgeving.

# Stap 4. Geografische Visualisatie van Materiaalhoeveelheden

In [4]:
import folium
from folium import IFrame

# Bepaal de kolomnamen voor latitude en longitude
lat_col = 'lat'
lon_col = 'lon'

kaart = folium.Map(location=[df_heerlen[lat_col].mean(), df_heerlen[lon_col].mean()], zoom_start=13)

for _, rij in df_heerlen.head(1000).iterrows():
    lat = rij[lat_col]
    lon = rij[lon_col]
    if pd.isna(lat) or pd.isna(lon):
        continue

    postcode = rij.get('postcode')
    huisnummer = rij.get('huisnummer')
    if pd.isna(postcode) or pd.isna(huisnummer):
        continue
    postcode_str = str(postcode)
    try:
        resultaat = bereken_materialen_per_adres(df_heerlen, df_materiaal, postcode_str, huisnummer)
    except Exception:
        continue

    if "resultaat" in resultaat and isinstance(resultaat["resultaat"], dict):
        materialen = resultaat["resultaat"]
        import plotly.graph_objects as go
        fig = go.Figure(data=[
            go.Bar(x=list(materialen.keys()), y=list(materialen.values()), marker_color='lightskyblue')
        ])
        fig.update_layout(
            title=f"Materialen voor {postcode_str} {huisnummer}<br>({resultaat.get('functie_categorie','')} - {resultaat.get('bouwjaar_interval','')})",
            height=300,
            margin=dict(l=0, r=0, t=30, b=0)
        )
        html = fig.to_html(include_plotlyjs='cdn')
    else:
        html = "<b>Geen gegevens beschikbaar</b>"

    iframe = IFrame(html, width=500, height=350)
    popup = folium.Popup(iframe, max_width=500)
    folium.Marker(
        location=[lat, lon],
        popup=popup,
        tooltip=f"{rij.get('functie', 'Onbekend')} ({postcode_str} {huisnummer})"
    ).add_to(kaart)

kaart

# Uitleg van de Code

De beschreven code maakt een interactieve kaart die gebruikmaakt van de folium-bibliotheek. Op deze kaart worden markeringen weergegeven voor de eerste 1000 gebouwen uit het df_heerlen-dataframe. Iedere markering heeft een pop-up die, waar mogelijk, een grafiek toont van de geschatte materialen voor dat specifieke adres.

1. Kaartinitialisatie:
De namen van de kolommen voor de breedte- en lengtegraad (lat_col en lon_col) worden vastgesteld.
Een folium. Map-object (de kaart) wordt gecreëerd. De kaart wordt gecentreerd op de gemiddelde breedte- en lengtegraden van de gebouwen in het df_heerlen-dataframe, met Heerlen als middelpunt. De startzoom is ingesteld op 13 om de stad gedetailleerd weer te geven. Iteratie over Gebouwen: De code doorloopt de eerste 1000 rijen van het df_heerlen-dataframe (df_heerlen. head(1000)). Dit is gedaan om de laadtijd te beperken en de kaart toegankelijk te houden, omdat het verwerken van een te groot aantal gebouwen met ingewikkelde pop-ups de prestaties negatief zou kunnen beïnvloeden. Bij elke rij (rij) worden de lat (breedtegraad) en lon (lengtegraad) opgehaald. Rijen zonder deze coördinaten worden overgeslagen. Daarnaast worden de postcode en het huisnummer geëxtraheerd. Ontbreken deze gegevens, dan wordt de huidige 
2. iteratie overgeslagen.
De functie bereken_materialen_per_adres() wordt voor elk gebouw aangeroepen om de geschatte hoeveelheden materialen te bepalen. Fouten die tijdens deze berekening optreden, worden stilzwijgend genegeerd, zodat de iteratie doorgaat naar het volgende gebouw.
3. Visualisatie van Materiaalgegevens in Pop-up:
Er wordt gecontroleerd of de resultaat-dictionary van bereken_materialen_per_adres() de sleutel "resultaat" bevat en of de waarde een niet-lege dictionary is, wat betekent dat de materiaaldataberekening succesvol was.
Als dat het geval is, wordt met plotly. graph_objects een staafdiagram (go. Bar) gemaakt. Dit diagram toont de verschillende materialen (x=list(materialen. keys())) en hun berekende hoeveelheden (y=list(materialen. values())). De lay-out van de grafiek wordt aangepast met een titel die informatie geeft over de postcode, het huisnummer, de functie en het bouwjaar van het gebouw. De hoogte en marges van de grafiek worden ingesteld zodat deze compact in de pop-up wordt weergegeven. De Plotly-grafiek wordt omgezet in een HTML-string (fig. to_html()), inclusief de vereiste Plotly JavaScript-bibliotheken via een CDN (Content Delivery Network). Dit garandeert dat de grafiek goed zichtbaar is in de Folium webomgeving. Wanneer er geen materiaalgegevens beschikbaar zijn, wordt er een eenvoudige HTML-bericht 'Geen gegevens beschikbaar' aangemaakt. Deze HTML-content wordt vervolgens geïntegreerd in een folium. IFrame-object, dat de inhoud op de webpagina in de pop-up toont. Een folium. Popup-object wordt aangemaakt waarin de IFrame is opgenomen en waarin een maximale breedte is ingesteld.
4. Marker Plaatsing op de Kaart:
Voor elk gebouw dat verwerkt is, wordt een folium. Marker geplaatst op de berekende locatie ([lat, lon]).
Deze marker krijgt de zojuist gecreëerde pop-up gekoppeld, zodat de materiaalgrafiek te zien is bij een klik op de marker. Er wordt een tooltip toegevoegd die de functie van het gebouw en het adres toont wanneer de muis over de marker beweegt. De marker wordt daarna aan de kaart toegevoegd.
5. Kaartweergave:
Uiteindelijk wordt het kaartobject getoond, wat leidt tot de interactieve wereldkaart met de aanduidingen en hun informatievensters.